In [43]:
#run pip install yfinance pandas --user before running this file 
from yahoo_fin.stock_info import get_live_price
from yahoo_fin import options as op
import pandas as pd
from datetime import datetime
import math
import random

In [44]:
#get contract and underlying information 
contractInfo = input("Enter the contract symbol for your option: ")
numContracts = int(input('Enter the number of contracts: '))
#AAPL240719C00105000 <- example contract that you can use as input
tickerSymbol = contractInfo[0:4]

underlyingPrice = get_live_price(tickerSymbol)

year = int(contractInfo[4:6])
month = int(contractInfo[6:8])
day = int(contractInfo[8:10])
expDate = datetime(2000 + year, month, day)

callOrPut = contractInfo[10]
strike = int(contractInfo[11:]) / 1000 

#get IV for option
def getIV():
  chain = []
  if callOrPut == 'C':
    chain = op.get_calls(tickerSymbol)
  else:
    chain = op.get_puts(tickerSymbol)
  
  chain = pd.DataFrame(chain)
  
  mask = chain['Strike'] == strike
  volPercent = chain[mask].head().to_numpy()[0][10]
  return volPercent[0:len(volPercent) - 1]

In [45]:
def getBlackScholesDelta():
  S = float(underlyingPrice) #underlying price
  K = float(strike) #strike
  r = 0.05 #risk free return 
  T = float((expDate - datetime.today()).days) #days to maturity
  vol = float(getIV()) / 100 #underlying vol
  
  #https://www.simtrade.fr/blog_simtrade/option-greeks-delta/
  delta = (math.log(S/K) + T * (r + math.pow(vol, 2) / 2)) / (vol * math.sqrt(T))
  
  if callOrPut == 'P':
    return 1 - delta
  return delta

In [46]:
history = [] #all purchases (adjustments) to the initial options purchase

def adjust(delta):
  if delta > 0:
    print(f"Sell {delta} shares of underlying")
    buySellUnderlying(-1 * delta)
  elif delta < 0:
    print(f"Buy {delta * -1} shares of underlying")
    buySellUnderlying(-1 * delta)
  else:
    print("No adjustments are required")

def buySellUnderlying(amount):
  history.append([amount, get_live_price(tickerSymbol)])
  #TODO: SEND AN EMAIL HERE SAYING HOW MUCH YOU WILL ADJUST BY
  
def getPositionDelta():
  #get delta of shares
  numShares = 0
  for purchase in history:
    numShares += purchase[0]
  
  #add shares delta to options delta
  return getBlackScholesDelta() * numContracts + numShares

In [47]:
#initial spread
optionDelta = getBlackScholesDelta()
print(f"Your initial spread is: ")
adjust(optionDelta)

Your initial spread is: 
Sell 12.261390303337778 shares of underlying


In [49]:
daysTillExpiration = (expDate - datetime.today()).days
threshold = float(input("Enter the threshold for the delta as decimal between 0.1 and 1(determines how frequently you will adjust): "))

#assuming one adjustment per day
for i in range(daysTillExpiration):   
  underlyingPrice = underlyingPrice + random.randint(-10, 10) #simulate new prices
  
  #check if delta has deviated from 0 by the threshold
  positionDelta = getPositionDelta()
  
  print(f"new delta: {positionDelta}")
  
  if abs(positionDelta) > threshold:
    adjust(positionDelta)

new delta: 0.07997422268349652
Sell 0.07997422268349652 shares of underlying
new delta: 0.014796445868221042
Sell 0.014796445868221042 shares of underlying
new delta: 0.007022844641170423
new delta: 0.011576892515222426
Sell 0.011576892515222426 shares of underlying
new delta: 0.004456866662806647
new delta: -0.013971314378025568
Buy 0.013971314378025568 shares of underlying
new delta: -0.02021284814705382
Buy 0.02021284814705382 shares of underlying
new delta: 0.0026361972157147306
new delta: -0.022380617551775117
Buy 0.022380617551775117 shares of underlying
new delta: 0.005823969515745375
new delta: -0.02176654662230959
Buy 0.02176654662230959 shares of underlying


KeyboardInterrupt: 